### Implementation of the codes developed, openTURNS style


In [1]:
import openturns as ot
import numpy as np
import PureBeamExample as pbe #Toy model to analyse
import KarhunenLoeveFieldSensitivity as klfs #Openturns integration of the analysis algorithm


Bad key "text.kerning_factor" on line 4 in
/home/simady/anaconda/envs/stochastic_field_env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In this notebook, we will show how the codes developed for the sensitivity anakysis on models governed by stochastic fields can be easily integrated in the actual openturns environment. 

We will use the toy example of the bending beam to demonstrate the method.

##### The behaviour of the algorithms is highly inspired of the ones found in the openTURNS API

###### The codes are composed of four parts that behave together :
    ot.AggregatedKarhunenLoeveResults
    ot.KarhunenLoeveSobolIndicesExperiment
    ot.KarhunenLoeveGeneralizedFunctionWrapper
    ot.SobolKarhunenLoeveFieldSensitivityAlgorithm
    

##### We are in the case where the model to analyse takes as an input mulitple stochastic fields and random variables. The model also returns multiple fields and scalar values. 

To be able to use random variables in our AggregatedKarhunenLoeveObject, we express our random variables as a constant stochastic field. In this way, we can still use our KarhunenLoeveDecomposition on it, but we will have to convert the constant field into a scalar value again just before passing it to our model. 
This must be coded manually. 

### Definition of the input processes and random variables :

In [2]:
# Function to convert a scalar distribution into a field : 

def variablesAsProcess(distribution, mesh):
    '''Function to transform a scalar distribution into 
    a constant process defined over a mesh
    '''
    basis = ot.Basis([ot.SymbolicFunction(['x'],['1'])])
    lawAsprocess = ot.FunctionalBasisProcess(distribution, basis, mesh)
    lawAsprocess.setName(distribution.getName())
    return lawAsprocess

#############################################################

# First step : DEFINING THE PROCESSES AND RANDOM VARIABLES
# First Process E_ : Youngs modulus
# dimension 
dimension = 1
#grid
#Number of elements:
NElem = [100]
mesher = ot.IntervalMesher(NElem)
lowBound = [0] #mm
highBound = [1000] #mm
interval = ot.Interval(lowBound,highBound)
mesh = mesher.build(interval)

#Covariance model Young's modulus
amplitude0 = [50000]*dimension
scale0 = [300]*dimension
nu0 = 13/3
model0 = ot.MaternModel(scale0, amplitude0, nu0)
# Karhunen Loeve decomposition of process 
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model0, 1e-3)
algorithm.run()
resultsE = algorithm.getResult()
resultsE.setName('E_')

# Second Process D_ : Diameter
amplitude = [.3]*dimension
scale = [250]*dimension
nu = 13/3
model1 = ot.MaternModel(scale, amplitude, nu)
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model1, 1e-3)
algorithm.run()
resultsD = algorithm.getResult()
resultsD.setName('D_')

#############################################################

# SECOND STEP : Definition of the random variables
# random variable for the density of the material (kg/m³)
sigma       = 750
nameD       = 'Rho'
RV_Rho = ot.Normal(0, sigma)
RV_Rho.setName(nameD)
# random variable for the position of the force   (mm) 
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ot.Normal(0, sigma_f)
RV_Fpos.setName(namePos)
# random variable for the norm of the force    (N)
sigma_Fnor    = 5.5
nameNor       = 'FN'
RV_Fnorm  = ot.Normal(0, sigma_Fnor)
RV_Fnorm.setName(nameNor)

#############################################################

# Then convert the distributions t processes over a mesh
SP_Rho = variablesAsProcess(RV_Rho, mesh)
SP_Fpos = variablesAsProcess(RV_Fpos, mesh)
SP_Fnorm = variablesAsProcess(RV_Fnorm, mesh)

#############################################################

# Then we can do the Karhunen Loeve decomposition of the distributions : 
algorithm0 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Rho.getCovarianceModel(), 1e-3)
algorithm0.run()
resultsRho = algorithm0.getResult()
resultsRho.setName('Rho_')

algorithm1 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Fpos.getCovarianceModel(), 1e-3)
algorithm1.run()
resultsFpos = algorithm1.getResult()
resultsFpos.setName('Fpos_')

algorithm2 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Fnorm.getCovarianceModel(), 1e-3)
algorithm2.run()
resultsFnor = algorithm2.getResult()
resultsFnor.setName('Fnorm_')

# Finally we get a list of aggregated KarhunenLoeve results
listOfKLRes = [resultsE, resultsD, resultsRho, resultsFpos, resultsFnor]


### NOTE :

When defining a distribution as a process, as shown above, it is of utmost importance to modifiy his function so that it takes as an input a constant process, not a distribution anymore. This can also be easily avoided, if you take the first value of the field generated.

###### Always multiple ways to proceed :

In the example presented here, 2 ways of approaching the problem are possible. First we can create all our processes individually, decompose them individually with Karhunen Loeve, and pass a list of KarhunenLoeveResults to our new function wrapper. 
But this is only necessary if we would have meshes of different shapes, or if we would have defined the scalar distributions on smaller meshes. But as all processes are defined over the same mesh here, we could have aggregated the Processes, and directly decompose the AggregatedProcess. 

###### Note2 :

When using the **KarhunenLoeveP1Algorithm** function, only the covariance function is decomposed this means **all data 'bout the means and the trend functions are FORGOTTEN**. You have to add these constants **Directly in yout function!** 


##### First tests with new architecture. 
our base is a list of Karhunen Loeve Results. This will be passed into a contructor called AggregatedKarhunenLoeveResults. This  class is necessary as it is used to pass between the one dimensional space of the KarhunenLoeve Coefficients to the arbitrary dimension of the field linked to it. 
This class keeps all the data about the decomposition and allows to work on lists of processes

In [3]:
AggregatedKLObj = klfs.AggregatedKarhunenLoeveResults(listOfKLRes)

In [4]:
AggregatedKLObj

class = AggregatedKarhunenLoeveResults| name = Unnamed| Aggregation Order = 5| Threshold = 0.001| Covariance Model 0 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[300] amplitude=class=Point name=Unnamed dimension=1 values=[50000] nu=4.33333| Covariance Model 1 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[250] amplitude=class=Point name=Unnamed dimension=1 values=[0.3] nu=4.33333| Covariance Model 2 = class=RankMCovarianceModel, variance=class=Point name=Unnamed dimension=1 values=[562500], covariance=class=CovarianceMatrix dimension=0 implementation=class=MatrixImplementation name=Unnamed rows=0 columns=0 values=[], basis=class=Basis coll=[class=Function name=Unnamed implementation=class=FunctionImplementation name=Unnamed description=[x,y0] evaluationImplementation=class=SymbolicEvaluation name=Unnamed inputVariablesNames=[x] outputVariablesNames=[y0] formulas=[1] gradientImplementation=class=SymbolicGradient name=Unnamed evaluation=c

### Now we initialize our toy example : 

In [5]:
## First: beam initialization with mesh
PureBeam = pbe.PureBeam(mesh)

### Then we pass our model into our function wrapper. We have two functions : 
- One for single evaluations
- One for batch evaluations
##### - Only one of the functions is required. 
#### In the wrapper we also pas the AggregatedKarhunenLoeveResults object, as well as the number of outputs the function has. 
## WARNING : 
#### It is not the dimension of the output (eg. the dimension of a field) but it represents the numer of different outputs the function has, as it is assumed that the function returns a tuple of scalars, fields etc. 

In [6]:
# Now that we have our function, we can pass it in our wrapper! 
FUNC = klfs.KarhunenLoeveGeneralizedFunctionWrapper(
                                AggregatedKarhunenLoeveResults = AggregatedKLObj,
                                func        = PureBeam.singleEval, 
                                func_sample = PureBeam.batchEval,
                                n_outputs   = 2)


#### Once the model is created as the AggregatedKarhunenLoeveResult object, we can begin our DOE (Design Of Experiment)
- For this, we use our class called **KarhunenLoeveSobolIndicesExperiment**. 
- This last class aims to be similar to the class ot.SobolIndicesExperiment. 
##### Generation of the input design : 

In [7]:
N = 2000
SobolExperiment = klfs.KarhunenLoeveSobolIndicesExperiment(AggregatedKLObj, N ,False)
inputDesign = SobolExperiment.generate()

Samples A and B of size 2000 and dimension 18
Experiment of size 14000 and dimension 18


##### Evaluation of the function on the input design

In [8]:
outputDesign = FUNC(inputDesign[:2000])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

shape deflection:  (2000, 103)  should be [N,10X] something
deflection std deviation  nan
Using the batch evaluation function. Assumes that the outputs are in the 
same order than for the single evaluation function. This one should only 
return ProcessSamples, Samples, Lists or numpy arrays.
Element is iterable, assumes that first dimension is size of sample
Shape is (2000, 102) and dtype is <class 'numpy.float64'>
Element 0 of the output tuple returns process samples of dimension 1
Element is iterable, assumes that first dimension is size of sample
Shape is (2000,) and dtype is <class 'numpy.float64'>
Element 1 of the output tuple returns samples of dimension 1


In [20]:
#Uncomment to save samples as csv
#import pandas as pd
#inp = inputDesign[:2000]
#out0 = outputDesign[0]
#out1 = outputDesign[1]
#inp.exportToCSVFile('./MetaModelSamples/sample2000_rand.csv')
#vonMises = ot.Sample(np.array(np.stack([np.squeeze(np.asarray(out0[i])) for i in range(len(out0))])))
#vonMises.exportToCSVFile('./MetaModelSamples/vonMises_rand.csv')
#out1.exportToCSVFile('./MetaModelSamples/maxDefl_rand.csv')


### Sensitivity  analysis part : 
##### We pass as an argument the estimator we want to use for the sensitivity analysis 

In [ ]:
FieldSensitivityAnalysis = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
FieldSensitivityAnalysis.setDesign(inputDesign, outputDesign, N)
FieldSensitivityAnalysis.setEstimator(ot.SaltelliSensitivityAlgorithm())

In [ ]:
FieldSensitivityAnalysis.getFirstOrderIndices()

In [ ]:
FieldSensitivityAnalysis.getFirstOrderIndicesInterval()

In [ ]:
FieldSensitivityAnalysis.getAggregatedFirstOrderIndices()

In [ ]:
FieldSensitivityAnalysis.getAggregatedTotalOrderIndices()

In [ ]:
FieldSensitivityAnalysis.getFirstOrderIndicesDistribution()

In [ ]:
FieldSensitivityAnalysis.getTotalOrderIndices()

In [ ]:
FieldSensitivityAnalysis.getTotalOrderIndicesDistribution()

In [ ]:
FieldSensitivityAnalysis.getTotalOrderIndicesInterval()

In [ ]:
FieldSensitivityAnalysis.getSecondOrderIndices()

In [ ]:
x = FieldSensitivityAnalysis.__results__[0]

In [ ]:
x

In [ ]:
gr = x.draw(2)

In [ ]:
gr.